# 01 Data Loading and Validation

**Purpose**  
Load core snapshot tables, normalize schema variations, validate taxonomy coverage, and assess data completeness before downstream analysis.

**Outputs preserved:** coverage statistics and missingness assessment.

## Configuration

In [ ]:
from pathlib import Path
import pandas as pd

from src.notebook_utils import load_data

PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "data").exists() and (PROJECT_ROOT.parent / "data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
DATA_PATH = PROJECT_ROOT / "data"
OUTPUT_PATH = PROJECT_ROOT / "outputs" / "figures"
TOP_N = 15
DATE_CANDIDATES = ["incident_date", "date", "incident_datetime", "incident_time"]

loaded_tables = load_data(
    DATA_PATH,
    tables=["incidents", "reports", "submissions", "mit", "gmf", "cset"],
)

incidents_df = loaded_tables["incidents"]
reports_df = loaded_tables["reports"]
submissions_df = loaded_tables["submissions"]
mit_df = loaded_tables["mit"]
gmf_df = loaded_tables["gmf"]
cset_df = loaded_tables["cset"]

if incidents_df is None or reports_df is None or submissions_df is None:
    raise FileNotFoundError("Required tables missing: incidents.csv, reports.csv, or submissions.csv.")

print("Incidents:", incidents_df.shape)
print("Reports:", reports_df.shape)
print("Submissions:", submissions_df.shape)
print("MIT:", None if mit_df is None else mit_df.shape)
print("GMF:", None if gmf_df is None else gmf_df.shape)
print("CSET:", None if cset_df is None else cset_df.shape)

## Schema Checks

In [ ]:
print("inc keys:", [c for c in incidents_df.columns if "incident" in c or c.endswith("_id")])
print("rep keys:", [c for c in reports_df.columns if "incident" in c or c.endswith("_id")])
print("sub keys:", [c for c in submissions_df.columns if "incident" in c or "url" in c or c.endswith("_id")])
print("mit keys:", [] if mit_df is None else [c for c in mit_df.columns if "incident" in c or c.endswith("_id")])

## Taxonomy Coverage

In [ ]:
def detect_incident_id_column(table_df: pd.DataFrame | None, label: str) -> str | None:
    if table_df is None:
        return None
    for candidate in ["incident_id", "incident id", "incidentid"]:
        if candidate in table_df.columns:
            return candidate
    for column_name in table_df.columns:
        if "incident" in column_name and "id" in column_name:
            return column_name
    print(f"[{label}] Incident id column not found.")
    return None

incident_id_values = set(incidents_df["incident_id"]) if "incident_id" in incidents_df.columns else set()

mit_incident_col = detect_incident_id_column(mit_df, "MIT")
gmf_incident_col = detect_incident_id_column(gmf_df, "GMF")
cset_incident_col = detect_incident_id_column(cset_df, "CSET")

print("MIT incident column:", mit_incident_col)
print("GMF incident column:", gmf_incident_col)
print("CSET incident column:", cset_incident_col)

def coverage_ratio(table_df: pd.DataFrame | None, incident_col: str | None, denominator: int) -> float:
    if table_df is None or incident_col is None or denominator == 0:
        return 0.0
    return table_df[incident_col].dropna().astype(str).nunique() / denominator

mit_coverage = coverage_ratio(mit_df, mit_incident_col, len(incident_id_values))
gmf_coverage = coverage_ratio(gmf_df, gmf_incident_col, len(incident_id_values))
cset_coverage = coverage_ratio(cset_df, cset_incident_col, len(incident_id_values))

print(f"MIT coverage:  {mit_coverage:.1%}")
print(f"GMF coverage:  {gmf_coverage:.1%}")
print(f"CSET coverage: {cset_coverage:.1%}")

print("Total reports (rows):", len(reports_df))
print("Unique report URLs:", reports_df["url"].nunique() if "url" in reports_df.columns else "n/a")
print("Unique source domains:", reports_df["source_domain"].nunique() if "source_domain" in reports_df.columns else "n/a")
print("Submissions rows (auxiliary):", len(submissions_df))
print("Note: No complete incident-report mapping table exists in this snapshot, so report counts are proxy-based.")

## Missingness Assessment

In [ ]:
missing_share = incidents_df.isna().mean().sort_values(ascending=False)
missing_share.head(25)

In [ ]:
high_missing_columns = missing_share[missing_share > 0.5]
print(f"Columns with >50% missingness: {len(high_missing_columns)}")
high_missing_columns

## Interpretation

- The notebook is deterministic and schema-aware (incident id detection handles common variants).
- Coverage rates are computed defensively and set to 0 when optional taxonomy tables are missing.
- Missingness diagnostics are preserved to support transparency before downstream analytics.